# 05: Adding Tools for Fact-Checking

Let's add search capabilities to verify claims with real data.

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from config.llm_factory import LLMFactory
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from typing import List
import json

print("✅ Ready to add tools!")

✅ Ready to add tools!


## Create a Search Tool

In [2]:
@tool
def search_aviation_facts(query: str) -> str:
    """Search for aviation facts to verify claims."""
    # Mock database of facts
    facts = {
        "boeing 747 engines": "The Boeing 747 has 4 engines, not 6",
        "wright brothers": "The Wright brothers first flew on December 17, 1903",
        "commercial altitude": "Commercial jets fly at 30,000-40,000 feet",
        "747": "The Boeing 747 has 4 engines mounted on the wings"
    }
    
    query_lower = query.lower()
    for key, fact in facts.items():
        if key in query_lower:
            return fact
    return "No specific information found."

tools = [search_aviation_facts]

## BS Detector with Tools

In [3]:
def check_claim_with_simple_tools(claim: str) -> dict:
    """Check claim using tools when needed"""
    llm = LLMFactory.create_llm()
    llm_with_tools = llm.bind_tools(tools)
    
    messages = [
        SystemMessage(content="""You are an aviation expert. 
        Use the search tool to verify factual claims.
        Respond with BS or LEGITIMATE based on evidence."""),
        HumanMessage(content=f"Check this claim: {claim}")
    ]
    
    # First LLM call - might request tool use
    response = llm_with_tools.invoke(messages)
    messages.append(response)
    
    # If tool was called, execute it
    if response.tool_calls:
        for tool_call in response.tool_calls:
            # Execute the tool
            if tool_call["name"] == "search_aviation_facts":
                result = search_aviation_facts.invoke(tool_call["args"])
                # Add tool result to messages
                tool_msg = ToolMessage(
                    content=result,
                    tool_call_id=tool_call["id"]
                )
                messages.append(tool_msg)
        
        # Get final response after tool use
        final_response = llm_with_tools.invoke(messages)
        content = final_response.content
    else:
        content = response.content
    
    # Parse response
    verdict = "BS" if "BS" in content.upper() else "LEGITIMATE"
    
    return {
        "verdict": verdict,
        "confidence": 85 if response.tool_calls else 70,
        "reasoning": content,
        "used_tools": bool(response.tool_calls)
    }

## Test the Tool-Enhanced Detector

In [4]:
# Test claims
test_claims = [
    "The Boeing 747 has six engines",
    "The Wright brothers first flew in 1903",
    "Commercial planes fly at 50,000 feet"
]

for claim in test_claims:
    print(f"\n🔍 Checking: {claim}")
    result = check_claim_with_simple_tools(claim)
    
    print(f"📊 Verdict: {result['verdict']} ({result['confidence']}%)")
    print(f"🔧 Used tools: {'Yes' if result['used_tools'] else 'No'}")
    print(f"💭 {result['reasoning'][:150]}...")


🔍 Checking: The Boeing 747 has six engines
📊 Verdict: BS (85%)
🔧 Used tools: Yes
💭 The claim that the Boeing 747 has six engines is BS. The Boeing 747 actually has four engines....

🔍 Checking: The Wright brothers first flew in 1903
📊 Verdict: LEGITIMATE (85%)
🔧 Used tools: Yes
💭 LEGITIMATE. The Wright brothers first flew on December 17, 1903....

🔍 Checking: Commercial planes fly at 50,000 feet
📊 Verdict: LEGITIMATE (85%)
🔧 Used tools: Yes
💭 The claim that commercial planes fly at 50,000 feet is generally not accurate. Typical cruising altitudes for commercial airliners are usually between...


## Use the Module Version

For more complex scenarios, use the full implementation:

In [5]:
from modules.m5_tools import check_claim_with_tools

# This version includes:
# - Web search via DuckDuckGo
# - Multi-agent routing
# - Better error handling

claim = "SpaceX launched 5 rockets yesterday"
result = check_claim_with_tools(claim)

print(f"Claim: {claim}")
print(f"Verdict: {result['verdict']} ({result['confidence']}%)")
print(f"Used search: {result.get('used_search', False)}")

/Users/nikpatel/Documents/GitHub/workshop-agents/.venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Claim: SpaceX launched 5 rockets yesterday
Verdict: BS (90%)
Used search: True


## Key Takeaways

1. **Tools enhance accuracy** - Search provides evidence
2. **LLM decides when to search** - Not every claim needs it
3. **Higher confidence** - Evidence-based verdicts
4. **Proper message flow** - Tool calls → Tool results → Final answer

Next: Add human-in-the-loop for uncertain cases!